In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.datasets import fetch_mldata, fetch_lfw_people
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from collections import defaultdict

## 2. Ensemble Learning
### (a) Load two text-based classiffication datasets (e.g., from NLTK or Kaggle) and pre- process as necessary to arrive at a numerical representation of the text. (Code)

## (a) [What's Cooking?](https://www.kaggle.com/c/whats-cooking)

In [21]:
cook_train = pd.read_json('cooking_train.json',orient='records').set_index('id')
cook_test = pd.read_json('cooking_test.json',orient='records').set_index('id')

In [22]:
cook_train.sample(5)

,cuisine,ingredients
id,,
23243,italian,"[sweet onion, salt, small red potato, dough, o..."
29455,japanese,"[green cardamom, milk, cinnamon sticks, conden..."
16876,greek,"[ground black pepper, purple onion, cucumber, ..."
47661,southern_us,"[flour, buttermilk, butter, baking powder, sal..."
38521,italian,"[black pepper, vegetable broth, boiling water,..."


In [23]:
cook_train['cuisine'].nunique()

20

In [24]:
cook_test.head(2)

,ingredients
id,
18009,"[baking powder, eggs, all-purpose flour, raisi..."
28583,"[sugar, egg yolks, corn starch, cream of tarta..."


In [89]:
# since there's no label for the test set, I'll split cooking_train into training and testing sets
cooking = cook_train
cooking.shape

(39774, 2)

In [90]:
cooking.dropna(axis=0, how='any', inplace=True)

In [91]:
# split "ingredient" into multiple columns
cooking = pd.concat([pd.DataFrame(cooking['ingredients'].values.tolist()), (cooking['cuisine'])], axis=1)
cooking.head(5)

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,cuisine
0,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,garbanzo beans,feta cheese crumbles,None,...,None,None,None,None,None,None,None,None,None,spanish
1,plain flour,ground pepper,salt,tomatoes,ground black pepper,thyme,eggs,green tomatoes,yellow corn meal,milk,...,None,None,None,None,None,None,None,None,None,mexican
2,eggs,pepper,salt,mayonaise,cooking oil,green chilies,grilled chicken breasts,garlic powder,yellow onion,soy sauce,...,None,None,None,None,None,None,None,None,None,french
3,water,vegetable oil,wheat,salt,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,chinese
4,black pepper,shallots,cornflour,cayenne pepper,onions,garlic paste,milk,butter,salt,lemon juice,...,None,None,None,None,None,None,None,None,None,italian


In [92]:
cooking.iloc[:,:65] = cooking.iloc[:,:65].fillna(value=0.0)
cooking.head(5)

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,cuisine
0,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,garbanzo beans,feta cheese crumbles,0,...,0,0,0,0,0,0,0,0,0,spanish
1,plain flour,ground pepper,salt,tomatoes,ground black pepper,thyme,eggs,green tomatoes,yellow corn meal,milk,...,0,0,0,0,0,0,0,0,0,mexican
2,eggs,pepper,salt,mayonaise,cooking oil,green chilies,grilled chicken breasts,garlic powder,yellow onion,soy sauce,...,0,0,0,0,0,0,0,0,0,french
3,water,vegetable oil,wheat,salt,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,chinese
4,black pepper,shallots,cornflour,cayenne pepper,onions,garlic paste,milk,butter,salt,lemon juice,...,0,0,0,0,0,0,0,0,0,italian


In [95]:
cooking.dropna(subset=['cuisine'], how='any', inplace=True)

In [138]:
cooking = cooking.sample(1000)   # sample 1000 from the dataset due to run time limit
cooking_X = cooking.iloc[:,:65]
cooking_Y = cooking['cuisine']

# sklearn.OneHotEncoder doesn't take string-based values
cooking_X = pd.get_dummies(cooking_X)

In [143]:
cooking_X.shape

(1000, 5003)

In [144]:
cooking_X_train, cooking_X_test, cooking_Y_train, cooking_Y_test = train_test_split(cooking_X, cooking_Y)
print("train shape: {}\ntest shape: {}".format(cooking_X_train.shape, cooking_X_test.shape))

train shape: (750, 5003)
test shape: (250, 5003)


In [145]:
sc = StandardScaler()
cooking_X_train_scaled = sc.fit_transform(cooking_X_train)
cooking_X_test_scaled = sc.transform(cooking_X_test)

le = LabelEncoder()
cooking_Y_train_enc = le.fit_transform(cooking_Y_train)
cooking_Y_test_enc = le.transform(cooking_Y_test)

In [147]:
%store cooking_X_train_scaled
%store cooking_X_test_scaled
%store cooking_Y_train_enc
%store cooking_Y_test_enc

Stored 'cooking_X_train_scaled' (ndarray)
Stored 'cooking_X_test_scaled' (ndarray)
Stored 'cooking_Y_train_enc' (ndarray)
Stored 'cooking_Y_test_enc' (ndarray)


## (a) [Mushroom Classification](https://www.kaggle.com/uciml/mushroom-classification)

In [2]:
mushrooms = pd.read_csv('mushrooms.csv')

In [3]:
mushrooms.shape

(8124, 23)

In [4]:
mushrooms['class'].nunique()

2

In [5]:
mushrooms.isnull().any()

class                       False
cap-shape                   False
cap-surface                 False
cap-color                   False
bruises                     False
odor                        False
gill-attachment             False
gill-spacing                False
gill-size                   False
gill-color                  False
stalk-shape                 False
stalk-root                  False
stalk-surface-above-ring    False
stalk-surface-below-ring    False
stalk-color-above-ring      False
stalk-color-below-ring      False
veil-type                   False
veil-color                  False
ring-number                 False
ring-type                   False
spore-print-color           False
population                  False
habitat                     False
dtype: bool

In [6]:
mushrooms.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [7]:
mushroom_X = mushrooms.iloc[:, 1:]
mushroom_Y = mushrooms['class']

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(pd.get_dummies(mushroom_X), mushroom_Y)

In [12]:
mushroom_d = defaultdict(LabelEncoder)
mushroom_X_fit = mushroom_X.apply(lambda x: mushroom_d[x.name].fit_transform(x))

In [13]:
le_Y = LabelEncoder()
mushroom_Y_fit = le_Y.fit_transform(mushroom_Y)

In [32]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# one hot encode column by column
def multicolumn_onehot_mushroom(X):
    
    onehot = defaultdict(OneHotEncoder)
    df = pd.DataFrame()

    for i in range(22):
        
        curr = pd.DataFrame(onehot[X.columns[i]].fit_transform(X.iloc[:,i:i+1]).toarray())
    
        oh = onehot[X.columns[i]]
        labelEncoder_i = mushroom_d[X.columns[i]]
        curr.columns = X.columns[i] + " " + labelEncoder_i.inverse_transform(oh.active_features_)
    
        X_onehot_i = curr.iloc[:,1:]

        df = pd.concat([df, X_onehot_i], axis=1)
    
    return df

In [33]:
mushroom_X_df = multicolumn_onehot_mushroom(mushroom_X_fit)

In [34]:
mushroom_X_df.head(10)

,cap-shape c,cap-shape f,cap-shape k,cap-shape s,cap-shape x,cap-surface g,cap-surface s,cap-surface y,cap-color c,cap-color e,...,population n,population s,population v,population y,habitat g,habitat l,habitat m,habitat p,habitat u,habitat w
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
mushroom_X_train, mushroom_X_test, mushroom_Y_train, mushroom_Y_test = train_test_split(mushroom_X_df.values, mushroom_Y_fit.reshape(-1,1))

sc = StandardScaler()
sc = sc.fit(mushroom_X_train)
mushroom_X_train_scaled = sc.transform(mushroom_X_train)
mushroom_X_test_scaled = sc.transform(mushroom_X_test)

## (b) For each dataset, evaluate these classiffiers using 10-fold cross validation: (Code)
### (i) Three different classifiers (e.g., SVM, Naive Bayes, Decision Tree, KNN).

In [49]:
def knn(X_train, X_test, Y_train, Y_test):
    
    # cross validate to select optimal K
    cv_scores = []
    kf = KFold(n_splits=10, shuffle=False, random_state=1)
    k_list = [5, 10, 20, 30, 50, 80, 120]
    
    for k in k_list:
        cv_model = KNeighborsClassifier(n_neighbors=k)
        score = np.mean(cross_val_score(cv_model, X_train, Y_train, cv=kf))
        cv_scores.append(score)
    cv_scores = np.asarray(cv_scores)
    print("10-fold cross-validated score on training set:", cv_scores)
    
    best_k = K_list[cv_scores.argmax()]
    print("Best K =", best_k)
    model = KNeighborsClassifier(n_neighbors=best_k)
    model.fit(X_train, Y_train)

    print("Best score on test set:", model.score(X_test, Y_test))


################################################################################


def logit(X_train, X_test, Y_train, Y_test):
    cv_scores = []
    kf = KFold(n_splits=10, shuffle=False, random_state=1)
    C_list = [0.5, 1.0, 2.5, 5.0]
    
    for c in C_list:
        cv_model = LogisticRegression(C=c, random_state=2, warm_start=True)
        score = np.mean(cross_val_score(cv_model, X_train, Y_train, cv=kf))
        cv_scores.append(score)
        
    cv_scores = np.asarray(cv_scores)
    
    print("10-fold cross-validated score on training set:", cv_scores)
    
    best_C = C_list[cv_scores.argmax()]
    print("Best C =", best_C)
    model = LogisticRegression(C=best_C, random_state=2, warm_start=True)
    model.fit(X_train, Y_train)
    
    print("Best score on test set:", model.score(X_test, Y_test))


################################################################################


def lda(X_train, X_test, Y_train, Y_test):
    
    kf = KFold(n_splits=10, shuffle=False, random_state=1)
    cv_score = np.mean(cross_val_score(LinearDiscriminantAnalysis(), X_train, Y_train, cv=kf))
    print("10-fold cross validation score on training set:", cv_score)
    
    model = LinearDiscriminantAnalysis()
    model.fit(X_train, Y_train)
    
    print("Best score on test set:", model.score(X_test, Y_test))

#### mushroom dataset

In [44]:
import warnings
warnings.filterwarnings("ignore")

logit(mushroom_X_train, mushroom_X_test, mushroom_Y_train, mushroom_Y_test)

10-fold cross-validated score on training set: [0.99901478 0.99934319 1.         1.         1.        ]
Best score on test set: 1.0


In [45]:
lda(mushroom_X_train_scaled, mushroom_X_test_scaled, mushroom_Y_train, mushroom_Y_test)

10-fold cross validation score on training set: 0.9991789819376027
Best score on test set: 1.0


In [48]:
knn(mushroom_X_train_scaled, mushroom_X_test_scaled, mushroom_Y_train, mushroom_Y_test)

KeyboardInterrupt: 

#### cooking dataset

In [ ]:
logit(cooking_X_train_scaled, cooking_X_test_scaled, cooking_Y_train_enc, cooking_Y_test_enc)

In [53]:
lda(cooking_X_train_scaled, cooking_X_test_scaled, cooking_Y_train_enc, cooking_Y_test_enc)

10-fold cross validation score on training set: 0.10266666666666664
Best score on test set: 0.176


In [ ]:
knn(cooking_X_train_scaled, cooking_X_test_scaled, cooking_Y_train_enc, cooking_Y_test_enc)

### ii. Majority vote classiffier that uses the three classiffiers from previous step.